In [1]:
import numpy as np
import matplotlib.pyplot as plt
from orix.quaternion import Orientation, symmetry
from orix.plot import IPFColorKeyTSL
from orix.vector import Vector3d

import utilities
import pyHREBSD

%matplotlib widget

In [2]:
# Load the scan data and print out some information
# up2 = "E:/cells/CoNi90-OrthoCells.up2"
# ang = "E:/cells/CoNi90-OrthoCells.ang"
# name = "CoNi90-OrthoCells"
up2 = "E:/cells/CoNi90-ParallelCells.up2"
ang = "E:/cells/CoNi90-ParallelCells.ang"
name = "CoNi90-ParallelCells"

pixel_size = 13.0
Nxy = (2048, 2048)

pat_obj, ang_data = utilities.get_scan_data(up2, ang, Nxy, pixel_size)
print("Scan data loaded")
print("File size:", pat_obj.filesize)
print("Pattern shape:", pat_obj.patshape)
print("Number of patterns:", pat_obj.nPatterns)
print("Scan shape:", ang_data.shape)
print("Ang fields:", ang_data._fields)

DoG_sigmas = (1.0, 10.0)

pats = utilities.get_patterns(pat_obj).reshape(ang_data.shape + pat_obj.patshape)
# utilities.test_bandpass(pats[0, 0])
pats_processed = utilities.process_patterns(pats, equalize=True, dog_sigmas=DoG_sigmas)
sharpness = utilities.get_sharpness(pats_processed)

Scan data loaded
File size: 5242.9 MB
Pattern shape: (256, 256)
Number of patterns: 40000
Scan shape: (200, 200)
Ang fields: ('x', 'y', 'iq', 'ci', 'phase_index', 'sem', 'fit', 'eulers', 'quats', 'shape', 'pc', 'pidx')


Reading patterns:   0%|          | 0/40000 [00:00<?, ?pats/s]

Processing patterns:   0%|          | 0/10000 [00:00<?, ?pats/s]

c:\Users\lambj\miniconda3\envs\hrebsd\Lib\site-packages\kornia\filters\filter.py:129: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  output = F.conv2d(input, tmp_kernel, groups=tmp_kernel.size(0), padding=0, stride=1)


Calculating sharpness:   0%|          | 0/40000 [00:00<?, ?pats/s]

In [ ]:
ipfkey100 = IPFColorKeyTSL(symmetry.Oh, Vector3d((1, 0, 0)))
ipf100 = ipfkey100.orientation2color(Orientation(ang_data.quats, symmetry.Oh))

utilities.view(utilities.shade_ipf(ipf100, sharpness))
plt.savefig(f"./results/{name}_IPF100-sharpness.png", dpi=300)
utilities.view(sharpness, cmap="gray")
plt.savefig(f"./results/{name}_sharpness.png", dpi=300)
utilities.view(ipf100)
plt.savefig(f"./results/{name}_IPF100.png", dpi=300)

In [ ]:
utilities.view(np.rot90(sharpness))

In [5]:
# Set the reference pattern and the subset slice to use for the HREBSD analysis
save_name = name + f"_R100-115"
R = pats_processed[100, 115]
subset_slice = (slice(10, -10), slice(10, -10))

In [6]:
# Calculate the initial guesses for the HREBSD analysis
p0 = pyHREBSD.get_initial_guess(R, pats_processed)
np.save(f"{save_name}_p0.npy", p0)
# p0 = np.load(f"{save_name}_p0.npy", p0)

There are enough targets to parallelize. Starting pool.


100%|██████████| 5714/5714 [29:21<00:00,  1.06s/it]


In [7]:
# Run the HREBSD analysis using the initial guesses
p = pyHREBSD.get_homography(R, pats_processed, subset_slice=subset_slice, p0=p0, max_iter=50, conv_tol=1e-7)
np.save(f"{save_name}_p.npy", p)
# p = np.load(f"{save_name}_p.npy")

IC-GN:   0%|          | 0/40000 [00:00<?, ?targets/s]

In [8]:
# Calculate the deviatoric deformation gradient tensor
Fe = pyHREBSD.homography_to_elastic_deformation(p, ang_data.pc)

# Calculate the strain tensor and the rotation tensor
e, w = pyHREBSD.deformation_to_strain(Fe)

In [15]:
import os


# Save out the results
xy = (100, 115) # The location of the reference point

plt.close('all')
utilities.view_tensor_images(np.rot90(Fe, axes=(0, 1)), tensor_type="deformation", xy=xy, save_name=save_name, save_dir="results/")
utilities.view_tensor_images(np.rot90(e, axes=(0, 1)), tensor_type="strain", xy=xy, save_name=save_name, save_dir="results/", show="upper")
utilities.view_tensor_images(np.rot90(w, axes=(0, 1)), tensor_type="rotation", xy=xy, save_name=save_name, save_dir="results/", show="upper")